In [1]:
# LLM React agent–Driven RNA Design via Structural Feedback and RNA Diffusion Model
# Do no forget to run container with DRfold2 model

import os, sys
from importlib import reload
sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../'))
from langchain_anthropic import ChatAnthropic
import json 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="torch")

# Choose llm model
#llm = ChatAnthropic(model="claude-3-5-sonnet-20241022")
#llm = ChatAnthropic(model="claude-3-haiku-20240307")
#llm = ChatAnthropic(model="claude-3-5-haiku-20241022")
#llm = ChatAnthropic(model="claude-3-7-sonnet-20250219")
llm = ChatAnthropic(model="claude-sonnet-4-20250514")

##https://console.anthropic.com/settings/billing
# Open‑source (GLM‑4.5 etc)



In [2]:
# FASTA file read/write
from lib.fasta_utils import read_single_fasta, write_single_fasta

# DRfold2 model
from lib.lib_drfold import fasta2pdb

# Ribodiffusion model
from lib.lib_ribodiffusion import Pdb2Fasta
pdb2fasta = Pdb2Fasta()

c:\Users\User\miniconda3\envs\rna_agents\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\work\rna_agent\RiboDiffusion\utils.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend yo

In [3]:
# Define the RNA evolution seed (starting RNA sequence for the simulation)
# Define RNA sequences to be used in the analysis

import yaml

# Open and load the JSON file
with open('../configs/rna_rag.yaml', 'r') as f:
    config = yaml.safe_load(f)

# input_dir = "../data/processed/rna_evolution_seed/"
# output_dir = "../data/processed/RNA_reports/"
input_dir = config['input_output']['input_dir']
output_dir = config['input_output']['output_dir_ReActAgent']
os.makedirs(output_dir, exist_ok=True)
# Or recreate folder (delete if exists)
# if os.path.exists(folder): shutil.rmtree(folder)
# os.makedirs(folder)

fasta_file = "trnaGlycine_Asgard_group_archaeon.fasta"
fasta_file = "simple_sequence.fasta"
sequence = read_single_fasta(os.path.join(input_dir, fasta_file))[1]

simple_sequence = sequence
tRNA = "GCACCGAUAGUCUAGUGGUAGGACUUAUCCCUGCCUAGGAUAGAGCCCGGGUUCAAAUCCCGGUCGGUGCACCA"
print(simple_sequence)
print(tRNA)

AGAGAGAGAGAGAGAGAGAG
GCACCGAUAGUCUAGUGGUAGGACUUAUCCCUGCCUAGGAUAGAGCCCGGGUUCAAAUCCCGGUCGGUGCACCA


In [4]:
# Open and load the JSON file
with open('../configs/rna_rag.yaml', 'r') as f:
    config = yaml.safe_load(f)
path = config['input_output']['output_dir_ReActAgent']

In [5]:
config['input_output']['input_dir']
config['input_output']['output_dir_ToolAgent']
#config['input_output']['output_dir_ReActAgent']

'../data/processed/rna_evolution/'

In [6]:
# Select tools for ReAct agent

import lib.agents.lib_RNAtools
reload(lib.agents.lib_RNAtools)

from lib.agents.lib_tools import biopython_pubmed_tool, search_tool

from lib.agents.lib_RNAtools import get_RNA_2alignment_tool
from lib.agents.lib_RNAtools import get_hairpins_tool, get_helices_tool, get_stems_tool
from lib.agents.lib_RNAtools import get_dotBracket_length_tool, get_multiplets_tool, get_junctions_tool
from lib.agents.lib_RNAtools import get_riboseZippers_tool, get_stacks_tool, get_nonStack_tool
from lib.agents.lib_RNAtools import get_numCoaxStacks_tool, get_backfoldedRNAsequence_tool

from lib.agents.lib_RNAtools import get_length_pairs_hydrogenBonds_tool

tools = [get_RNA_2alignment_tool]
tools += [get_hairpins_tool, get_helices_tool, get_stems_tool]
tools += [get_dotBracket_length_tool, get_multiplets_tool, get_junctions_tool]
tools += [get_riboseZippers_tool, get_stacks_tool, get_nonStack_tool]
tools += [get_numCoaxStacks_tool, get_backfoldedRNAsequence_tool]
tools += [get_length_pairs_hydrogenBonds_tool]

tools += [biopython_pubmed_tool, search_tool]
model = llm.bind_tools(tools)

c:\Users\User\miniconda3\envs\rna_agents\Lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [7]:
# Test some tools
# it takes some time to calculate 3D structure for the given sequence
print(get_hairpins_tool.func(tRNA))
print(get_helices_tool.func(tRNA))
print(get_stems_tool.func(tRNA))
print(get_dotBracket_length_tool.func(tRNA))
print(get_multiplets_tool.func(tRNA))
print(get_junctions_tool.func(tRNA))
print(get_riboseZippers_tool.func(tRNA))
print(get_stacks_tool.func(tRNA)) 
print(get_nonStack_tool.func(tRNA))
print(get_numCoaxStacks_tool.func(tRNA))
print(get_backfoldedRNAsequence_tool.func(tRNA))
print(get_length_pairs_hydrogenBonds_tool.func(tRNA))

UAGUGGUAG, CCUGCCUAG, GUUCAAAUC
helix_0: base_pairs=13, strand_1=GCACCGACCGGGU, strand_2=CGUGGCUGGCCCA, helix_form=AAAAAAxAAAAx
helix_1: base_pairs=13, strand_1=CAGGAUAGUCUAG, strand_2=UCCCUAUCAGGUC, helix_form=x.AAAAxAA.xx
helix_0: base_pairs=7, strand_1=GCACCGA, strand_2=CGUGGCU, helix_form=AAAAAA
helix_1: base_pairs=4, strand_1=GUCU, strand_2=CAGG, helix_form=AA.
helix_2: base_pairs=5, strand_1=UAUCC, strand_2=AUAGG, helix_form=AAAA
helix_3: base_pairs=5, strand_1=CCGGG, strand_2=GGCCC, helix_form=AAAA
2D RNA structure in dot-bracket notation: (((((((..((((.......)))).(((((.......)))))....(((((.......)))))))))))).... RNA length: 74 nucleotides
UAC, UGC, ACG, UGG
AUAGCUUAGAGCCGU
UAAG
AC, UU, GC, CA, AC, GU, CUU, CCU, AGA, GUU, CAC, UGAG, CUAG, AGACAU
GGUCA
2
GGUCCGAUGGUGUAGUGGUAUCACACAGCCCUAGGAAGGCUGGCACCCAGGUUCAAAUCCUGGUCGGACCACCA
Length: 74 nucleotides, Number of base pairs: 33, Number of hydrogen bonds: 123.


In [8]:
# Start LangChain Graph construction

from typing import Annotated, Sequence, TypedDict
# from langchain_core.messages import HumanMessage
from langchain_core.messages import BaseMessage, SystemMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

class AgentState(TypedDict): 
    messages: Annotated[Sequence[BaseMessage], add_messages]
    #messages: List[Union[HumanMessage, AIMessage]]


In [9]:
# ReAct agent
def model_call(state:AgentState) -> AgentState:
    content = (
        "You are a research assistant tasked with conducting RNA evolutionary research. "
        "Your goal is to analyze RNA structures and simulate evolutionary modifications. "
        "Use the necessary tools when required."
    )
    system_prompt = SystemMessage(content=content)
    response = model.invoke([system_prompt]+state["messages"])
    return {"messages": [response]}

# Define nodes    
graph = StateGraph(AgentState)
graph.add_node("our_agent", model_call)
tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

In [10]:
# Set edges
graph.set_entry_point("our_agent")
def should_continue(state:AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"
graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    }
)

graph.add_edge("tools", "our_agent")
app = graph.compile()

def print_stream(stream):
    # Configure proper stdout output
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [11]:
print(simple_sequence)
print(tRNA)

AGAGAGAGAGAGAGAGAGAG
GCACCGAUAGUCUAGUGGUAGGACUUAUCCCUGCCUAGGAUAGAGCCCGGGUUCAAAUCCCGGUCGGUGCACCA


In [12]:
# Run Inference for the given query

query = (
    f"Current tRNA sequence (input): {tRNA}\n\n"
    "Modify this reference sequence by increasing the length of the acceptor stem to **twice its original size**. "
    "Ensure that **no more than 20%** of the nucleotides are changed or added.\n\n"
    "Keep the rest of the tRNA unchanged.\n\n"
    "Verify the structure and return the modified RNA sequence."
)

query = (
    f"Provide a detailed comparison of the following two RNA molecules: query {simple_sequence} and target {tRNA}.\n\n"
    "1. Run a simple sequence alignment between them.\n"
    "2. Analyze and compare their secondary and tertiary structures.\n"
    "3. Conduct a comprehensive search of scientific literature to identify known information about these sequences.\n"
    "4. Highlight their similarities and differences.\n\n"
    "Provide the output in clearly labeled sections."
)

query = (
    f"What is a ribonucleic acid (RNA) molecule?\n\n"
)

inputs = {"messages": [("user", query)]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

What is a ribonucleic acid (RNA) molecule?


================================== Ai Message ==================================

A ribonucleic acid (RNA) molecule is a fundamental biological macromolecule that plays crucial roles in various cellular processes. Here's a comprehensive overview:

## Basic Structure

RNA is a nucleic acid composed of:
- **Ribose sugar** (a 5-carbon sugar with a hydroxyl group on the 2' carbon)
- **Phosphate groups** that form the backbone
- **Four nitrogenous bases**: Adenine (A), Guanine (G), Cytosine (C), and Uracil (U) - note that RNA contains uracil instead of thymine found in DNA

## Key Characteristics

1. **Single-stranded**: Unlike DNA, RNA is typically single-stranded, allowing it to fold into complex three-dimensional structures
2. **Less stable**: The 2'-OH group makes RNA more chemically reactive and less stable than DNA
3. **Shorter**: RNA molecules are generally s

In [13]:
remove
# Run this to generate a PDB file from the sequence generated by the ReAct agent, if needed
# Manually plug this sequence in here

generatedSequence = "GCACCGAUAGUCUAGUGGUAGAAAAGGACUUAUCCCUGCCUAGGAUAGAGCCCGGGUUCAAAUCCCGGUGCACCA"

import lib.fasta_utils
reload(lib.fasta_utils)
from lib.fasta_utils import read_single_fasta, write_single_fasta
fasta_generatedSequence = input_dir + "tRNAgenerated" + ".fasta"
write_single_fasta(header="seq", sequence=generatedSequence, filepath=fasta_generatedSequence)

print("- Calculating PDB from FASTA") 
pdb_path = fasta2pdb(source_path = fasta_generatedSequence)
# The PDB file is in the same directory as the FASTA file

NameError: name 'remove' is not defined

In [ ]:
fasta_file_next